## run this cell before starting spark cluster connection (corrects module error)

In [1]:
# spark.jars.packages com.databricks:spark-avro_2.11:4.0.0
import os
print os.environ['PYSPARK_PYTHON']
os.environ['PYSPARK_PYTHON']=os.environ['LCG_VIEW'] + '/bin/python'
print os.environ['PYSPARK_PYTHON']

/etc/spark/python
/cvmfs/sft.cern.ch/lcg/views/LCG_93/x86_64-slc6-gcc62-opt/bin/python


### Import necessary spark and python stuff

In [2]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.functions import from_json
from pyspark.ml.feature import VectorAssembler

In [3]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', -1)

In [4]:
df_lookup = spark.read.json("/user/sprenner/lookup_final_count.json")

In [5]:
df_lookup.count()

12361

In [6]:
df_april = spark.read.json("/user/sprenner/training_official_april_cleaned_datasets.json")

In [7]:
df_april = df_april.withColumn("join_key_april", concat_ws(':', df_april["datasetScope"], df_april["dataset"]))
df_april.createOrReplaceTempView("april")

In [8]:
df_april.limit(5).toPandas()

account  accountHash clientState  clientStateHash  \
0  hegormse -2036406190   DONE       -878527175         
1  vkitali   1625481889   DONE       -878527175         
2  mproffit  1290301367   DONE       -878527175         
3  mproffit  1290301367   DONE       -878527175         
4  mproffit  1290301367   DONE       -878527175         

                                                                                 dataset  \
0  data15_13TeV.00280862.physics_Main.deriv.DAOD_EXOT22.r9264_p3083_p3402_tid12752857_00   
1  data15_13TeV.00284285.physics_Main.deriv.DAOD_EXOT5.r9264_p3083_p3481_tid13613132_00    
2  data15_13TeV.00267073.physics_Main.deriv.DAOD_EXOT8.r9264_p3083_p3481_tid13466289_00    
3  data15_13TeV.00267162.physics_Main.deriv.DAOD_EXOT8.r9264_p3083_p3481_tid13466294_00    
4  data15_13TeV.00276181.physics_Main.deriv.DAOD_EXOT8.r9412_p3083_p3481_tid13466382_00    

   datasetHash  datasetScope  datasetScopeHash  \
0  1268366167   data15_13TeV  1217185487         
1 -106274841    data15_13TeV  1217185487         
2 -1055546401   data15_13TeV  1217185487         
3 -1582503124   data15_13TeV  1217185487         
4 -1817287115   data15_13TeV  1217185487         

                    dataset_version  dataset_versionHash     datatype  \
0  r9264_p3083_p3402_tid12752857_00  1724430760           DAOD_EXOT22   
1  r9264_p3083_p3481_tid13613132_00 -1787190451           DAOD_EXOT5    
2  r9264_p3083_p3481_tid13466289_00  772360213            DAOD_EXOT8    
3  r9264_p3083_p3481_tid13466294_00  2105334052           DAOD_EXOT8    
4  r9412_p3083_p3481_tid13466382_00 -959087507            DAOD_EXOT8    

   datatypeHash eventType  eventTypeHash eventVersion  eventVersionHash  \
0 -141841045     download  59378080       1.14.11     -2044360925         
1  1547296573    download  59378080       1.14.11     -2044360925         
2 -754389316     download  59378080       1.14.11     -2044360925         
3 -754389316     download  59378080       1.14.11     -2044360925         
4 -754389316     download  59378080       1.14.11     -2044360925         

      fileScope                                  filename  filenameHash  \
0  data15_13TeV  DAOD_EXOT22.12752857._000007.pool.root.1 -1554238082     
1  data15_13TeV  DAOD_EXOT5.13613132._000112.pool.root.1  -2031820614     
2  data15_13TeV  DAOD_EXOT8.13466289._000043.pool.root.1  -286165056      
3  data15_13TeV  DAOD_EXOT8.13466294._000153.pool.root.1  -1251159597     
4  data15_13TeV  DAOD_EXOT8.13466382._000031.pool.root.1   1916844419     

   filesize_double  hits               hostname  hostnameHash prod_step  \
0  6.236237e+08     1     lxplus104.cern.ch      494806480     deriv      
1  2.173912e+09     1     nafhh-atlas03.desy.de -492212293     deriv      
2  1.038860e+07     1     lxplus065.cern.ch      922885429     deriv      
3  1.381898e+07     1     lxplus065.cern.ch      922885429     deriv      
4  2.010680e+07     1     lxplus061.cern.ch     -1008250248    deriv      

   prod_stepHash       project  projectHash protocol  protocolHash  \
0 -1058503065     data15_13TeV  1217185487   srm      863299160      
1 -1058503065     data15_13TeV  1217185487   srm      863299160      
2 -1058503065     data15_13TeV  1217185487   srm      863299160      
3 -1058503065     data15_13TeV  1217185487   srm      863299160      
4 -1058503065     data15_13TeV  1217185487   srm      863299160      

              remoteSite  remoteSiteHash run_number  run_numberHash  \
0  TOKYO-LCG2_DATADISK   -2113133296      00280862   1717963924       
1  AGLT2_DATADISK        -1643637004      00284285   1039245728       
2  FMPHI-UNIBA_DATADISK   1301399448      00267073  -360919798        
3  UKI-LT2-RHUL_DATADISK  1896750258      00267162  -2019438915       
4  TOKYO-LCG2_DATADISK   -2113133296      00276181  -1147842459       

    scopeHash   stream_name  stream_nameHash  \
0  1217185487  physics_Main  680461412         
1  1217185487  physics_Main  680461412         
2  1217185487  physics_

In [9]:
df_april.count()

777286

In [10]:
df_joined = df_april.join(df_lookup, df_april.join_key_april == df_lookup.join_key_lookup, 'left_outer')
df_joined.count()

777286

In [11]:
df_joined.limit(10).toPandas()

account  accountHash         clientState  clientStateHash  \
0  hegormse -2036406190   DONE               -878527175         
1  vkitali   1625481889   DONE               -878527175         
2  mproffit  1290301367   DONE               -878527175         
3  mproffit  1290301367   DONE               -878527175         
4  mproffit  1290301367   DONE               -878527175         
5  mproffit  1290301367   DONE               -878527175         
6  yuchengh  1175694101   DONE               -878527175         
7  sosen     627971258    DONE               -878527175         
8  yuchengh  1175694101   ServiceUnavailable -1394468120        
9  yuchengh  1175694101   ServiceUnavailable -1394468120        

                                                                                  dataset  \
0  data15_13TeV.00280862.physics_Main.deriv.DAOD_EXOT22.r9264_p3083_p3402_tid12752857_00    
1  data15_13TeV.00284285.physics_Main.deriv.DAOD_EXOT5.r9264_p3083_p3481_tid13613132_00     
2  data15_13TeV.00267073.physics_Main.deriv.DAOD_EXOT8.r9264_p3083_p3481_tid13466289_00     
3  data15_13TeV.00267162.physics_Main.deriv.DAOD_EXOT8.r9264_p3083_p3481_tid13466294_00     
4  data15_13TeV.00276181.physics_Main.deriv.DAOD_EXOT8.r9412_p3083_p3481_tid13466382_00     
5  data15_13TeV.00276262.physics_Main.deriv.DAOD_EXOT8.r9264_p3083_p3481_tid13466393_00     
6  data15_13TeV.00276329.physics_Main.deriv.DAOD_HIGG3D1.r9264_p3083_p3388_tid12645178_00   
7  data15_13TeV.00276329.physics_Main.deriv.DAOD_HIGG3D1.r9264_p3083_p3388_tid12645178_00   
8  data15_13TeV.00280231.physics_Main.deriv.DAOD_HIGG3D1.r9264_p3083_p3388_tid12645374_00   
9  data15_13TeV.00280231.physics_Main.deriv.DAOD_HIGG3D1.r9264_p3083_p3388_tid12645374_00   

   datasetHash  datasetScope  datasetScopeHash  \
0  1268366167   data15_13TeV  1217185487         
1 -106274841    data15_13TeV  1217185487         
2 -1055546401   data15_13TeV  1217185487         
3 -1582503124   data15_13TeV  1217185487         
4 -1817287115   data15_13TeV  1217185487         
5 -1586092532   data15_13TeV  1217185487         
6 -1195931820   data15_13TeV  1217185487         
7 -1195931820   data15_13TeV  1217185487         
8  49121971     data15_13TeV  1217185487         
9  49121971     data15_13TeV  1217185487         

                    dataset_version  dataset_versionHash      datatype  \
0  r9264_p3083_p3402_tid12752857_00  1724430760           DAOD_EXOT22    
1  r9264_p3083_p3481_tid13613132_00 -1787190451           DAOD_EXOT5     
2  r9264_p3083_p3481_tid13466289_00  772360213            DAOD_EXOT8     
3  r9264_p3083_p3481_tid13466294_00  2105334052           DAOD_EXOT8     
4  r9412_p3083_p3481_tid13466382_00 -959087507            DAOD_EXOT8     
5  r9264_p3083_p3481_tid13466393_00  154324744            DAOD_EXOT8     
6  r9264_p3083_p3388_tid12645178_00 -1685856037           DAOD_HIGG3D1   
7  r9264_p3083_p3388_tid12645178_00 -1685856037           DAOD_HIGG3D1   
8  r9264_p3083_p3388_tid12645374_00 -403546695            DAOD_HIGG3D1   
9  r9264_p3083_p3388_tid12645374_00 -403546695            DAOD_HIGG3D1   

   datatypeHash eventType  eventTypeHash eventVersion  eventVersionHash  \
0 -141841045     download  59378080       1.14.11     -2044360925         
1  1547296573    download  59378080       1.14.11     -2044360925         
2 -754389316     download  59378080       1.14.11     -2044360925         
3 -754389316     download  59378080       1.14.11     -2044360925         
4 -754389316     download  59378080       1.14.11     -2044360925         
5 -754389316     download  59378080       1.14.11     -2044360925         
6 -499511313     download  59378080       1.14.11     -2044360925         
7 -499511313     download  59378080       1.14.11     -2044360925         
8 -499511313     download  59378080       1.14.11     -2044360925         
9 -499511313     download  59378080       1.14.11     -2044360925         

      fileScope                                   filename  filenameHash

## check

In [12]:
df_joined.createOrReplaceTempView("joined")
spark.sql("SELECT * FROM joined WHERE join_key_lookup IS NULL").toPandas()

account  accountHash         clientState  clientStateHash  \
0     schsu    -927799723    DONE               -878527175         
1     schsu    -927799723    ServiceUnavailable -1394468120        
2     toyamaza  368790896    DONE               -878527175         
3     toyamaza  368790896    DONE               -878527175         
4     sgargiul  1343070029   DONE               -878527175         
5     sgargiul  1343070029   DONE               -878527175         
6     sgargiul  1343070029   DONE               -878527175         
7     sgargiul  1343070029   DONE               -878527175         
8     sgargiul  1343070029   DONE               -878527175         
9     aknue     720293188    DONE               -878527175         
10    lhavener  1677896849   DONE               -878527175         
11    toyamaza  368790896    DONE               -878527175         
12    toyamaza  368790896    DONE               -878527175         
13    toyamaza  368790896    DONE               -878527175         
14    toyamaza  368790896    DONE               -878527175         
15    toyamaza  368790896    DONE               -878527175         
16    sgargiul  1343070029   DONE               -878527175         
17    toyamaza  368790896    DONE               -878527175         
18    toyamaza  368790896    DONE               -878527175         
19    toyamaza  368790896    DONE               -878527175         
20    toyamaza  368790896    DONE               -878527175         
21    sgargiul  1343070029   DONE               -878527175         
22    sgargiul  1343070029   DONE               -878527175         
23    sgargiul  1343070029   ServiceUnavailable -1394468120        
24    sgargiul  1343070029   DONE               -878527175         
25    toyamaza  368790896    DONE               -878527175         
26    toyamaza  368790896    DONE               -878527175         
27    toyamaza  368790896    DONE               -878527175         
28    toyamaza  368790896    DONE               -878527175         
29    sgargiul  1343070029   DONE               -878527175         
...        ...         ...    ...                      ...         
1594  toyamaza  368790896    DONE               -878527175         
1595  toyamaza  368790896    DONE               -878527175         
1596  toyamaza  368790896    DONE               -878527175         
1597  lcorpe   -279559867    DONE               -878527175         
1598  willocq   1840148994   DONE               -878527175         
1599  sgargiul  1343070029   DONE               -878527175         
1600  sgargiul  1343070029   DONE               -878527175         
1601  sgargiul  1343070029   DONE               -878527175         
1602  sgargiul  1343070029   DONE               -878527175         
1603  aknue     720293188    DONE               -878527175         
1604  sgargiul  1343070029   DONE               -878527175         
1605  sgargiul  1343070029   ServiceUnavailable -1394468120        
1606  sgargiul  1343070029   DONE               -878527175         
1607  willocq   1840148994   DONE               -878527175         
1608  therwig  -83317217     DONE               -878527175         
1609  toyamaza  368790896    DONE               -878527175         
1610  toyamaza  368790896    DONE               -878527175         
1611  toyamaza  368790896    DONE               -878527175         
1612  toyamaza  368790896    DONE               -878527175         
1613  toyamaza  368790896    DONE               -878527175         
1614  toyamaza  368790896    DONE               -878527175         
1615  toyamaza  368790896    DONE               -878527175         
1616  willocq   1840148994   DONE               -878527175         
1617  lhavener  1677896849   DONE               -878527175         
1618  toyamaza  368790896    DONE               -878527175         
1619  sgargiul  1343070029   DONE               -878527175         
1620  aknue     720293188    DONE               -878527175     

## which datasets downloaded by multiple accounts?

In [13]:
spark.sql("SELECT dataset, COUNT(DISTINCT account) AS accounts FROM joined GROUP BY dataset ORDER BY accounts DESC").toPandas()

dataset  \
0      mc16_13TeV.361239.Pythia8EvtGen_A3NNPDF23LO_minbias_inelastic_high.simul.HITS.e4981_s3087_s3111_tid10701335_00                                             
1      mc16_13TeV.364137.Sherpa_221_NNPDF30NNLO_Ztautau_MAXHTPTV280_500_CVetoBVeto.merge.NTUP_PILEUP.e5307_s3126_r9364_r9315_p3127_p3126_tid11325585_00           
2      mc16_13TeV.364137.Sherpa_221_NNPDF30NNLO_Ztautau_MAXHTPTV280_500_CVetoBVeto.merge.NTUP_PILEUP.e5307_s3126_r9364_r9315_p3127_p3126_tid11326065_00           
3      mc16_13TeV.361238.Pythia8EvtGen_A3NNPDF23LO_minbias_inelastic_low.simul.HITS.e4981_s3087_s3111_tid10701323_00                                              
4      mc16_13TeV.364106.Sherpa_221_NNPDF30NNLO_Zmumu_MAXHTPTV140_280_CVetoBVeto.merge.NTUP_PILEUP.e5271_s3126_r9364_r9315_p3127_p3126_tid11327067_00             
5      mc16_13TeV.361106.PowhegPythia8EvtGen_AZNLOCTEQ6L1_Zee.merge.NTUP_PILEUP.e3601_s3126_r9364_r9315_p3127_p3126_tid11326650_00                                
6      mc16_13TeV.364128.Sherpa_221_NNPDF30NNLO_Ztautau_MAXHTPTV0_70_CVetoBVeto.merge.NTUP_PILEUP.e5307_s3126_r9364_r9315_p3127_p3126_tid11325477_00              
7      mc16_13TeV.364134.Sherpa_221_NNPDF30NNLO_Ztautau_MAXHTPTV140_280_CVetoBVeto.merge.NTUP_PILEUP.e5307_s3126_r9364_r9315_p3127_p3126_tid11325549_00           
8      mc16_13TeV.364134.Sherpa_221_NNPDF30NNLO_Ztautau_MAXHTPTV140_280_CVetoBVeto.merge.NTUP_PILEUP.e5307_s3126_r9364_r9315_p3127_p3126_tid11326053_00           
9      data17_13TeV.00339070.physics_EnhancedBias.merge.HIST.r10385_r10424_p2944_p2944_tid13694860_00                                                             
10     mc16_13TeV.364128.Sherpa_221_NNPDF30NNLO_Ztautau_MAXHTPTV0_70_CVetoBVeto.merge.NTUP_PILEUP.e5307_s3126_r9364_r9315_p3127_p3126_tid11326011_00              
11     mc16_13TeV.364106.Sherpa_221_NNPDF30NNLO_Zmumu_MAXHTPTV140_280_CVetoBVeto.merge.NTUP_PILEUP.e5271_s3126_r9364_r9315_p3127_p3126_tid11326870_00             
12     mc16_13TeV.361106.PowhegPythia8EvtGen_AZNLOCTEQ6L1_Zee.merge.NTUP_PILEUP.e3601_s3126_r9364_r9315_p3127_p3126_tid11326726_00                                
13     mc16_13TeV.410470.PhPy8EG_A14_ttbar_hdamp258p75_nonallhad.deriv.NTUP_PILEUP.e6337_e5984_s3126_r10201_r10210_p3384_tid13225128_00                           
14     mc16_13TeV.364100.Sherpa_221_NNPDF30NNLO_Zmumu_MAXHTPTV0_70_CVetoBVeto.merge.NTUP_PILEUP.e5271_s3126_r9364_r9315_p3127_p3126_tid11327043_00                
15     mc16_13TeV.364103.Sherpa_221_NNPDF30NNLO_Zmumu_MAXHTPTV70_140_CVetoBVeto.merge.NTUP_PILEUP.e5271_s3126_r9364_r9315_p3127_p3126_tid11326858_00              
16     mc16_13TeV.364109.Sherpa_221_NNPDF30NNLO_Zmumu_MAXHTPTV280_500_CVetoBVeto.merge.NTUP_PILEUP.e5271_s3126_r9364_r9315_p3127_p3126_tid11326882_00             
17     mc16_13TeV.364135.Sherpa_221_NNPDF30NNLO_Ztautau_MAXHTPTV140_280_CFilterBVeto.merge.NTUP_PILEUP.e5307_s3126_r9364_r9315_p3127_p3126_tid11325561_00         
18     mc16_13TeV.364140.Sherpa_221_NNPDF30NNLO_Ztautau_MAXHTPTV500_1000.merge.NTUP_PILEUP.e5307_s3126_r9364_r9315_p3127_p3126_tid11326077_00                     
19     mc16_13TeV.364100.Sherpa_221_NNPDF30NNLO_Zmumu_MAXHTPTV0_70_CVetoBVeto.merge.NTUP_PILEUP.e5271_s3126_r9364_r9315_p3127_p3126_tid11326846_00                
20     mc16_13TeV.364113.Sherpa_221_NNPDF30NNLO_Zmumu_MAXHTPTV1000_E_CMS.merge.NTUP_PILEUP.e5271_s3126_r9364_r9315_p3127_p3126_tid11326906_00                     
21     mc16_13TeV.364107.Sherpa_221_NNPDF30NNLO_Zmumu_MAXHTPTV140_280_CFilterBVeto.merge.NTUP_PILEUP.e5271_s3126_r9364_r9315_p3127_p3126_tid11326874_00           
22     mc16_13TeV.364138.Sherpa_221_NNPDF30NNLO_Ztautau_MAXHTPTV280_500_CFilterBVeto.merge.NTUP_PILEUP.e5313_s3126_r9364_r9315_p3127_p3126_tid11325597_00         
23     mc16_13TeV.364132.Sherpa_221_NNPDF30NNLO_Ztautau_MAXHTPTV70_140_CFilterBVeto.merge.NTUP_PILEUP.e5307_s3126_r9364_r9315_p3127_p3126_tid11325525_00          
24     mc16_13TeV.364110.Sherpa_221_NNPDF30NNLO_Zmumu_MAXHTPTV280_500_CFilter

## how many different uuids for a certain dataset?

In [14]:
spark.sql("SELECT uuid, COUNT(*) AS count \
           FROM joined WHERE dataset=='mc16_13TeV.364137.Sherpa_221_NNPDF30NNLO_Ztautau_MAXHTPTV280_500_CVetoBVeto.merge.NTUP_PILEUP.e5307_s3126_r9364_r9315_p3127_p3126_tid11325585_00'\
           GROUP BY uuid \
           ORDER BY count DESC").toPandas()

uuid  count
0  4fe142912a384413b718294604c0c4d8  2    
1  b5c2dc21c7b84b419b9ec3d17a21e91d  2    
2  80c3bd43279f4478a5212139e883c322  1    
3  1d7be20eecb0437a98d608bee6a7dbb7  1    
4  b325b949079e49b3a8cd9a92b8216f12  1    
5  67f5aae3ecf0434cb312b5eda69e6d91  1    
6  c3bc604eada14d9886bdb87defaa0fa2  1    
7  bc3ebed807364c209626133b95a95b87  1

In [15]:
spark.sql("SELECT account,accountHash,clientState,clientStateHash,dataset,datasetHash,datasetScope,datasetScopeHash,dataset_version,dataset_versionHash,datatype,datatypeHash,eventType,eventTypeHash,eventVersion,eventVersionHash,fileScope,filename,filenameHash,filesize_double,hits,hostname,hostnameHash,prod_step,prod_stepHash,project,projectHash,protocol,protocolHash,remoteSite,remoteSiteHash,run_number,run_numberHash,scopeHash,stream_name,stream_nameHash,traceId,traceIdHash,traceIp,traceIpHash,traceTimeentryUnix,transferStart,uuid,uuidHash,join_key_april,count,join_key_lookup \
           FROM joined \
           WHERE uuid == 'e0b095f04c014ab583084cb07eb64dd4'").toPandas()

account  accountHash         clientState  clientStateHash  \
0  schsu  -927799723    DONE               -878527175         
1  schsu  -927799723    ServiceUnavailable -1394468120        

                                                                                                                                 dataset  \
0  mc16_13TeV.304124.MadGraphPythia8EvtGen_A14NNPDF23LO_zp2hdm_bb_mzp1400_mA700.deriv.DAOD_HIGG5D1.e4429_e5984_s3126_r10201_r10210_p3490   
1  mc16_13TeV.304124.MadGraphPythia8EvtGen_A14NNPDF23LO_zp2hdm_bb_mzp1400_mA700.deriv.DAOD_HIGG5D1.e4429_e5984_s3126_r10201_r10210_p3490   

   datasetHash datasetScope  datasetScopeHash  \
0 -1883788030   mc16_13TeV  -1447489496         
1 -1883788030   mc16_13TeV  -1447489496         

                         dataset_version  dataset_versionHash      datatype  \
0  e4429_e5984_s3126_r10201_r10210_p3490 -2086134313           DAOD_HIGG5D1   
1  e4429_e5984_s3126_r10201_r10210_p3490 -2086134313           DAOD_HIGG5D1   

   datatypeHash eventType  eventTypeHash eventVersion  eventVersionHash  \
0  1133870593    download  59378080       1.14.11     -2044360925         
1  1133870593    download  59378080       1.14.11     -2044360925         

    fileScope                                   filename  filenameHash  \
0  mc16_13TeV  DAOD_HIGG5D1.13652466._000004.pool.root.1 -247263268      
1  mc16_13TeV  DAOD_HIGG5D1.13652466._000004.pool.root.1 -247263268      

   filesize_double  hits         hostname  hostnameHash prod_step  \
0  996959837.0      2     pdsf8.nersc.gov  851963399     deriv      
1  996959837.0      2     pdsf8.nersc.gov  851963399     deriv      

   prod_stepHash     project  projectHash protocol  protocolHash  \
0 -1058503065     mc16_13TeV -1447489496   gsiftp   2128034789     
1 -1058503065     mc16_13TeV -1447489496   root    -1364943180     

           remoteSite  remoteSiteHash run_number  run_numberHash   scopeHash  \
0  CERN-PROD_DATADISK -2057286004      304124     965960893      -1447489496   
1  CERN-PROD_DATADISK -2057286004      304124     965960893      -1447489496   

                                                  stream_name  \
0  MadGraphPythia8EvtGen_A14NNPDF23LO_zp2hdm_bb_mzp1400_mA700   
1  MadGraphPythia8EvtGen_A14NNPDF23LO_zp2hdm_bb_mzp1400_mA700   

   stream_nameHash                           traceId  traceIdHash  \
0  962716114        842e75805f034a5fb942b59b71df2758  1349218702    
1  962716114        008f2a0f67874186a7b6bb90fdb222e5  878108346     

                traceIp  traceIpHash  traceTimeentryUnix  transferStart  \
0  ::ffff:128.55.202.69  1200015204   1.524797e+09        1.524797e+09    
1  ::ffff:128.55.202.69  1200015204   1.524797e+09        1.524797e+09    

                               uuid    uuidHash  \
0  e0b095f04c014ab583084cb07eb64dd4 -2145630535   
1  e0b095f04c014ab583084cb07eb64dd4 -2145630535   

                                                                                                                                     join_key_april  \
0  mc16_13TeV:mc16_13TeV.304124.MadGraphPythia8EvtGen_A14NNPDF23LO_zp2hdm_bb_mzp1400_mA700.deriv.DAOD_HIGG5D1.e4429_e5984_s3126_r10201_r10210_p3490   
1  mc16_13TeV:mc16_13TeV.304124.MadGraphPythia8EvtGen_A14NNPDF23LO_zp2hdm_bb_mzp1400_mA700.deriv.DAOD_HIGG5D1.e4429_e5984_s3126_r10201_r10210_p3490   

  count join_key_lookup  
0  None  None            
1  None  None

In [16]:
#SELECT account,accountHash,clientState,clientStateHash,dataset,datasetHash,datasetScope,datasetScopeHash,dataset_version,dataset_versionHash,datatype,datatypeHash,eventType,eventTypeHash,eventVersion,eventVersionHash,fileScope,filename,filenameHash,filesize_double,hits,hostname,hostnameHash,prod_step,prod_stepHash,project,projectHash,protocol,protocolHash,remoteSite,remoteSiteHash,run_number,run_numberHash,scopeHash,stream_name,stream_nameHash,traceId,traceIdHash,traceIp,traceIpHash,traceTimeentryUnix,transferStart,uuid,uuidHash,join_key_april,count,join_key_lookup

## List downloads on certain dataset ordered by Time

In [17]:
spark.sql("SELECT account,uuid, from_unixtime(traceTimeentryUnix, 'y-MM-dd HH:mm:ss') AS date,join_key_april,count,join_key_lookup \
           FROM joined \
           WHERE dataset == 'mc16_13TeV.361238.Pythia8EvtGen_A3NNPDF23LO_minbias_inelastic_low.simul.HITS.e4981_s3087_s3111_tid10701323_00'\
           ORDER BY date ASC").toPandas()

account                              uuid                 date  \
0    martindl  8765542f2a3a456c8a9fa1da5cf7608d  2018-04-07 11:31:35   
1    janus     a6df290c2c824fdcba04760281a4c35e  2018-04-10 08:31:15   
2    janus     a6df290c2c824fdcba04760281a4c35e  2018-04-10 08:32:22   
3    janus     a6df290c2c824fdcba04760281a4c35e  2018-04-10 08:33:02   
4    janus     a6df290c2c824fdcba04760281a4c35e  2018-04-10 08:34:44   
5    janus     a6df290c2c824fdcba04760281a4c35e  2018-04-10 08:36:49   
6    janus     a6df290c2c824fdcba04760281a4c35e  2018-04-10 08:36:59   
7    janus     a6df290c2c824fdcba04760281a4c35e  2018-04-10 08:38:00   
8    janus     a6df290c2c824fdcba04760281a4c35e  2018-04-10 08:41:49   
9    janus     a6df290c2c824fdcba04760281a4c35e  2018-04-10 08:42:32   
10   janus     a6df290c2c824fdcba04760281a4c35e  2018-04-10 08:53:14   
11   nigillwa  0917a844fbbc47cebcc9e54653ff52c1  2018-04-10 09:35:03   
12   nigillwa  0917a844fbbc47cebcc9e54653ff52c1  2018-04-10 09:36:04   
13   nigillwa  0917a844fbbc47cebcc9e54653ff52c1  2018-04-10 09:40:21   
14   nigillwa  0917a844fbbc47cebcc9e54653ff52c1  2018-04-10 09:43:13   
15   nigillwa  0917a844fbbc47cebcc9e54653ff52c1  2018-04-10 09:48:49   
16   nigillwa  0917a844fbbc47cebcc9e54653ff52c1  2018-04-10 09:49:41   
17   nigillwa  0917a844fbbc47cebcc9e54653ff52c1  2018-04-10 09:54:23   
18   nigillwa  0917a844fbbc47cebcc9e54653ff52c1  2018-04-10 09:54:30   
19   nigillwa  0917a844fbbc47cebcc9e54653ff52c1  2018-04-10 10:00:32   
20   nigillwa  0917a844fbbc47cebcc9e54653ff52c1  2018-04-10 10:01:10   
21   nigillwa  0917a844fbbc47cebcc9e54653ff52c1  2018-04-10 10:02:00   
22   nigillwa  0917a844fbbc47cebcc9e54653ff52c1  2018-04-10 10:03:15   
23   nigillwa  0917a844fbbc47cebcc9e54653ff52c1  2018-04-10 10:06:09   
24   nigillwa  0917a844fbbc47cebcc9e54653ff52c1  2018-04-10 10:10:35   
25   nigillwa  0917a844fbbc47cebcc9e54653ff52c1  2018-04-10 10:11:26   
26   nigillwa  0917a844fbbc47cebcc9e54653ff52c1  2018-04-10 10:14:55   
27   nigillwa  0917a844fbbc47cebcc9e54653ff52c1  2018-04-10 10:14:55   
28   nigillwa  0917a844fbbc47cebcc9e54653ff52c1  2018-04-10 10:15:14   
29   nigillwa  0917a844fbbc47cebcc9e54653ff52c1  2018-04-10 10:24:05   
..        ...                               ...                  ...   
87   yoyamagu  8b1db50a884248728b5b3be8c6bb5016  2018-04-19 18:57:33   
88   yoyamagu  8b1db50a884248728b5b3be8c6bb5016  2018-04-19 18:58:07   
89   yoyamagu  8b1db50a884248728b5b3be8c6bb5016  2018-04-19 19:01:32   
90   yoyamagu  8b1db50a884248728b5b3be8c6bb5016  2018-04-19 19:02:56   
91   yoyamagu  8b1db50a884248728b5b3be8c6bb5016  2018-04-19 19:04:32   
92   yoyamagu  8b1db50a884248728b5b3be8c6bb5016  2018-04-19 19:05:20   
93   yoyamagu  8b1db50a884248728b5b3be8c6bb5016  2018-04-19 19:05:31   
94   yoyamagu  8b1db50a884248728b5b3be8c6bb5016  2018-04-19 19:08:05   
95   yoyamagu  8b1db50a884248728b5b3be8c6bb5016  2018-04-19 19:10:38   
96   yoyamagu  8b1db50a884248728b5b3be8c6bb5016  2018-04-19 19:11:14   
97   yoyamagu  8b1db50a884248728b5b3be8c6bb5016  2018-04-19 19:14:17   
98   yoyamagu  8b1db50a884248728b5b3be8c6bb5016  2018-04-19 19:18:02   
99   yoyamagu  8b1db50a884248728b5b3be8c6bb5016  2018-04-19 19:20:43   
100  yoyamagu  8b1db50a884248728b5b3be8c6bb5016  2018-04-19 19:24:53   
101  oda       5dbaa42cd31441199c5ce2a640d9d550  2018-04-21 20:01:40   
102  oda       5dbaa42cd31441199c5ce2a640d9d550  2018-04-21 20:05:50   
103  oda       5dbaa42cd31441199c5ce2a640d9d550  2018-04-21 20:08:19   
104  oda       5dbaa42cd31441199c5ce2a640d9d550  2018-04-21 20:12:23   
105  oda       5dbaa42cd31441199c5ce2a640d9d550  2018-04-21 20:25:21   
106  oda       5dbaa42cd31441199c5ce2a640d9d550  2018-04-21 22:02:05   
107  oda       5dbaa42cd31441199c5ce2a640d9d550  2018-04-21 22:02:06   
108  oda       5dbaa42cd31441199c5ce2a640d9d550  2018-04-21 22:02:08   
109  oda       ce0d94fccc5e45389595c9e2a7cae1ff  2018-04-21 22:04:01   
110  oda       5dbaa42cd31441199c5ce2a640d9d55

## check whether there are uuids assigned to multiple accounts

In [18]:
spark.sql("SELECT uuid, COUNT(DISTINCT account) AS count\
           FROM joined \
           GROUP BY uuid\
           ORDER BY count DESC").toPandas()

uuid  count
0      d937eec5a2754c978632fa3bf6cc9f17  1    
1      65599a0be58843178b0f9e17fbca3b8e  1    
2      02737f5a24ca42c4a9ddbf6b178c91e4  1    
3      a99fa11f2c3a4d7786b5400dc82797bf  1    
4      0bc5e6aa7adc4d9caacacdab5aa51b4b  1    
5      5ccdc80e8a5542508aef1c5c730e23ec  1    
6      4b2ae518f0eb4a5c9785e251a4a880e1  1    
7      ce359900e27f4ea0b3f93e9692ed6abd  1    
8      5214fc5f317d454b98574cc5ed461ce5  1    
9      4842e1d9d45044deb442be284cd304e3  1    
10     efa89e3b7b424a7685f8764bcd27da46  1    
11     c60d1db8a48e4c988dce2ce9cc47a318  1    
12     dac026511f884cdea19a0b1b95d0f7fb  1    
13     65929c2b9ee644d39f6e8e397697cf5c  1    
14     1e5493c3041343939a7c14c62beee88a  1    
15     0deed350f4184c21ad2d5f6db9214970  1    
16     a06450588a364bc5a1a6a30e6c102920  1    
17     2104377d18224e5daff20ddf71a69e2f  1    
18     d8e9f75680c34f9994dbf4eecc26aa43  1    
19     df592478884148af940c317b9667eefa  1    
20     fbc0d1421cea4936b1bb93495b00810d  1    
21     18bb358c86764ae5a74f3eb6f0b2e539  1    
22     495544afb8a94d7db87695dbcacdbcf7  1    
23     67726d4f35024ba7a07b3f6eb4a39f5b  1    
24     991f280c48c44d6c8ac9fd42205c9d03  1    
25     04952cab55ae48418173349f06e383fe  1    
26     353935a0546444c9a9e4cecaec11eab7  1    
27     f9d4ed58e890437e9e97b2a7a3e7a95f  1    
28     c39abe61d0c0479386ed5e6a9caf36ac  1    
29     87d79664af394c4d9400d4b1c2a29152  1    
...                                 ... ..    
12847  a887830fd8b3495cab80f3b5e7bd72ff  1    
12848  757829c3fdb947c0b655f613962bad59  1    
12849  efcc79374cfa45e4be6e57500f4f9d7d  1    
12850  64df98f602f9431c9bb44f3ce951087e  1    
12851  419e9912dfad4a19a11977d8f2b30660  1    
12852  0bf60a0733db4b6fa635f62a46cb4db1  1    
12853  b16d2fef50114e709edcedd57033c8dc  1    
12854  39a6fa75accb45f6836a7c8be6cb58f3  1    
12855  b36d174fc3524ea6a8eb9d670b928fa0  1    
12856  4e9b41800693448da0cfaa5f2e18111b  1    
12857  a1c690863a0d4ef9a8b5d04b7fadb0ec  1    
12858  ce48f160de73412fb9749d5d1287f574  1    
12859  61e7553728094693825efd355b8211b6  1    
12860  da494a92e520451fa094cc9cbe9494d6  1    
12861  09983e584f304a8b96a6077acea22044  1    
12862  9af9b7a476184475bf1881242ed1f06a  1    
12863  ee00838cfa1a4bc49741d4c462412d60  1    
12864  17e318fccb934207a50e533354d88e03  1    
12865  63873bfb4be349bf9385cafbef5598d6  1    
12866  17fa8b533ecd48dea87258a677e12b3a  1    
12867  963bd70d4f6044149596ca0026330e98  1    
12868  7c11c621cf9c4b2fb6855e8fdd2addb6  1    
12869  b1b1721c846d4074a7407f8f60a01e55  1    
12870  f672919a48644bac94eac9f9aedf4e15  1    
12871  270eea83f46442488cc88a39314a67b7  1    
12872  56a2e82f63894c158833cdc73f52be7c  1    
12873  8e0c90684e2149a9986b5ab2673748db  1    
12874  a4cf722e39604584bbaf1f6ae2083723  1    
12875  16ae96d810784ed8891eb468aabc7161  1    
12876  f5065cdade174f3fb993449e742936e2  1    

[12877 rows x 2 columns]

In [19]:
spark.sql("SELECT * FROM joined ORDER BY traceTimeentryUnix ASC").limit(50).toPandas()

account  accountHash         clientState  clientStateHash  \
0   yliu      1494332205   ServiceUnavailable -1394468120        
1   yliu      1494332205   DONE               -878527175         
2   yliu      1494332205   ServiceUnavailable -1394468120        
3   changqia  224190018    DONE               -878527175         
4   changqia  224190018    ServiceUnavailable -1394468120        
5   yliu      1494332205   ServiceUnavailable -1394468120        
6   yliu      1494332205   DONE               -878527175         
7   heng      99410464     DONE               -878527175         
8   yliu      1494332205   DONE               -878527175         
9   yliu      1494332205   DONE               -878527175         
10  yliu      1494332205   DONE               -878527175         
11  yliu      1494332205   ServiceUnavailable -1394468120        
12  yliu      1494332205   ServiceUnavailable -1394468120        
13  yliu      1494332205   DONE               -878527175         
14  yliu      1494332205   ServiceUnavailable -1394468120        
15  yliu      1494332205   SourceNotFound     -1913538301        
16  yliu      1494332205   ServiceUnavailable -1394468120        
17  yliu      1494332205   DONE               -878527175         
18  heng      99410464     DONE               -878527175         
19  yliu      1494332205   ServiceUnavailable -1394468120        
20  yliu      1494332205   DONE               -878527175         
21  yliu      1494332205   DONE               -878527175         
22  yliu      1494332205   DONE               -878527175         
23  yliu      1494332205   ServiceUnavailable -1394468120        
24  yliu      1494332205   ServiceUnavailable -1394468120        
25  yliu      1494332205   ServiceUnavailable -1394468120        
26  yliu      1494332205   ServiceUnavailable -1394468120        
27  yliu      1494332205   ServiceUnavailable -1394468120        
28  changqia  224190018    DONE               -878527175         
29  ssolomon  1221542610   ServiceUnavailable -1394468120        
30  ssolomon  1221542610   DONE               -878527175         
31  changqia  224190018    ServiceUnavailable -1394468120        
32  changqia  224190018    ServiceUnavailable -1394468120        
33  changqia  224190018    ServiceUnavailable -1394468120        
34  changqia  224190018    ServiceUnavailable -1394468120        
35  ssolomon  1221542610   ServiceUnavailable -1394468120        
36  ssolomon  1221542610   DONE               -878527175         
37  yliu      1494332205   DONE               -878527175         
38  yliu      1494332205   DONE               -878527175         
39  ssolomon  1221542610   ServiceUnavailable -1394468120        
40  ssolomon  1221542610   DONE               -878527175         
41  yliu      1494332205   DONE               -878527175         
42  ssolomon  1221542610   ServiceUnavailable -1394468120        
43  ssolomon  1221542610   DONE               -878527175         
44  yliu      1494332205   DONE               -878527175         
45  yliu      1494332205   ServiceUnavailable -1394468120        
46  yliu      1494332205   ServiceUnavailable -1394468120        
47  yliu      1494332205   ServiceUnavailable -1394468120        
48  yliu      1494332205   DONE               -878527175         
49  yliu      1494332205   DONE               -878527175         

                                                                                                                                           dataset  \
0   data15_13TeV.00280231.physics_Main.deriv.DAOD_HIGG5D1.r9264_p3083_p3213_tid11568050_00                                                           
1   data15_13TeV.00280231.physics_Main.deriv.DAOD_HIGG5D1.r9264_p3083_p3213_tid11568050_00                                                           
2   data15_13TeV.00280231.physics_Main.deriv.DAOD_HIGG5D1.r9264_p3083_p3213_tid11568050_00                                                           
3   data16_13TeV.00300655.physics_Main

In [20]:
spark.sql("SELECT DISTINCT clientState FROM joined").toPandas()

clientState
0   RSEAccessDenied   
1   DONE              
2   CannotAuthenticate
3   Exception         
4   ServiceUnavailable
5   SourceNotFound    
6   FAIL_VALIDATE     
7   RucioException    
8   MissingDependency 
9   OSError           
10  GError

## create index column to later join the partition table correctly

In [21]:
df_joined_indexed = df_joined.sort("traceTimeentryUnix", ascending=True)
df_joined_indexed.createOrReplaceTempView("joined_indexed")
df_joined_indexed = spark.sql("select ROW_NUMBER() OVER (ORDER BY traceTimeentryUnix) as id, * FROM joined_indexed")

In [22]:
df_joined_indexed.limit(10).toPandas()

id   account  accountHash         clientState  clientStateHash  \
0  1   yliu      1494332205   ServiceUnavailable -1394468120        
1  2   yliu      1494332205   DONE               -878527175         
2  3   yliu      1494332205   ServiceUnavailable -1394468120        
3  4   changqia  224190018    DONE               -878527175         
4  5   changqia  224190018    ServiceUnavailable -1394468120        
5  6   yliu      1494332205   ServiceUnavailable -1394468120        
6  7   yliu      1494332205   DONE               -878527175         
7  8   heng      99410464     DONE               -878527175         
8  9   yliu      1494332205   DONE               -878527175         
9  10  yliu      1494332205   DONE               -878527175         

                                                                                                                                          dataset  \
0  data15_13TeV.00280231.physics_Main.deriv.DAOD_HIGG5D1.r9264_p3083_p3213_tid11568050_00                                                           
1  data15_13TeV.00280231.physics_Main.deriv.DAOD_HIGG5D1.r9264_p3083_p3213_tid11568050_00                                                           
2  data15_13TeV.00280231.physics_Main.deriv.DAOD_HIGG5D1.r9264_p3083_p3213_tid11568050_00                                                           
3  data16_13TeV.00300655.physics_Main.deriv.DAOD_HIGG5D1.r9264_p3083_p3372_tid12610361_00                                                           
4  data16_13TeV.00300655.physics_Main.deriv.DAOD_HIGG5D1.r9264_p3083_p3372_tid12610361_00                                                           
5  data15_13TeV.00280231.physics_Main.deriv.DAOD_HIGG5D1.r9264_p3083_p3213_tid11568050_00                                                           
6  data15_13TeV.00280231.physics_Main.deriv.DAOD_HIGG5D1.r9264_p3083_p3213_tid11568050_00                                                           
7  mc16_13TeV.345056.PowhegPythia8EvtGen_NNPDF3_AZNLO_ZH125J_MINLO_vvbb_VpT.deriv.DAOD_HIGG5D1.e5706_e5984_s3126_r9364_r9315_p3260_tid12139571_00   
8  data15_13TeV.00280231.physics_Main.deriv.DAOD_HIGG5D1.r9264_p3083_p3213_tid11568050_00                                                           
9  data15_13TeV.00280231.physics_Main.deriv.DAOD_HIGG5D1.r9264_p3083_p3213_tid11568050_00                                                           

   datasetHash  datasetScope  datasetScopeHash  \
0 -477538451    data15_13TeV  1217185487         
1 -477538451    data15_13TeV  1217185487         
2 -477538451    data15_13TeV  1217185487         
3 -654722309    data16_13TeV -915656957          
4 -654722309    data16_13TeV -915656957          
5 -477538451    data15_13TeV  1217185487         
6 -477538451    data15_13TeV  1217185487         
7  3665910      mc16_13TeV   -1447489496         
8 -477538451    data15_13TeV  1217185487         
9 -477538451    data15_13TeV  1217185487         

                                      dataset_version  dataset_versionHash  \
0  r9264_p3083_p3213_tid11568050_00                   -762138301             
1  r9264_p3083_p3213_tid11568050_00                   -762138301             
2  r9264_p3083_p3213_tid11568050_00                   -762138301             
3  r9264_p3083_p3372_tid12610361_00                    2032569103            
4  r9264_p3083_p3372_tid12610361_00                    2032569103            
5  r9264_p3083_p3213_tid11568050_00                   -762138301             
6  r9264_p3083_p3213_tid11568050_00                   -762138301             
7  e5706_e5984_s3126_r9364_r9315_p3260_tid12139571_00  747979904             
8  r9264_p3083_p3213_tid11568050_00                   -762138301             
9  r9264_p3083_p3213_tid11568050_00                   -762138301             

       datatype  datatypeHash eventType  eventTypeHash eventVersion  \
0  DAOD_HIGG5D1  1133870593    download  59378080       1.14.11       
1  DAOD_HIGG5D1  1133870593    download  59378080  

In [23]:
df_joined_indexed.createOrReplaceTempView("joined_indexed")

In [24]:
df_joined_indexed.count()

777286

## partition by 2 columns to count amount of files already downloaded in every dataset (uuid alone not enough, collisions).
## only count successful downloads to avoid >100% (STILL POSSIBLE if user downloads data twice)

In [25]:
#https://www.sqlservercentral.com/Forums/573706/ROWNUMBER-and-possibly-group-by
#https://stackoverflow.com/questions/6127338/sql-mysql-select-distinct-unique-but-return-all-columns
df_proportion = spark.sql("SELECT ROW_NUMBER() OVER (PARTITION BY uuid, join_key_april ORDER BY traceTimeentryUnix ASC) AS Seq , *\
                           FROM (SELECT * FROM \
                                    (SELECT *, ROW_NUMBER() OVER(PARTITION BY uuid, join_key_april, filename ORDER BY traceTimeentryUnix ASC) AS row_number\
                                     FROM joined_indexed WHERE clientState == 'DONE' AND join_key_lookup IS NOT NULL)\
                                 WHERE row_number == 1) \
                           ORDER BY traceTimeentryUnix ASC")
df_proportion.limit(50).toPandas()

Seq   id   account  accountHash clientState  clientStateHash  \
0   1    2    yliu      1494332205   DONE       -878527175         
1   1    4    changqia  224190018    DONE       -878527175         
2   2    7    yliu      1494332205   DONE       -878527175         
3   1    8    heng      99410464     DONE       -878527175         
4   3    9    yliu      1494332205   DONE       -878527175         
5   4    10   yliu      1494332205   DONE       -878527175         
6   5    11   yliu      1494332205   DONE       -878527175         
7   6    14   yliu      1494332205   DONE       -878527175         
8   7    18   yliu      1494332205   DONE       -878527175         
9   2    19   heng      99410464     DONE       -878527175         
10  8    21   yliu      1494332205   DONE       -878527175         
11  9    22   yliu      1494332205   DONE       -878527175         
12  10   23   yliu      1494332205   DONE       -878527175         
13  2    29   changqia  224190018    DONE       -878527175         
14  1    31   ssolomon  1221542610   DONE       -878527175         
15  1    37   ssolomon  1221542610   DONE       -878527175         
16  11   38   yliu      1494332205   DONE       -878527175         
17  12   39   yliu      1494332205   DONE       -878527175         
18  1    41   ssolomon  1221542610   DONE       -878527175         
19  13   42   yliu      1494332205   DONE       -878527175         
20  1    44   ssolomon  1221542610   DONE       -878527175         
21  14   45   yliu      1494332205   DONE       -878527175         
22  15   49   yliu      1494332205   DONE       -878527175         
23  16   50   yliu      1494332205   DONE       -878527175         
24  3    52   heng      99410464     DONE       -878527175         
25  17   53   yliu      1494332205   DONE       -878527175         
26  3    55   changqia  224190018    DONE       -878527175         
27  1    57   yliu      1494332205   DONE       -878527175         
28  4    59   changqia  224190018    DONE       -878527175         
29  5    64   changqia  224190018    DONE       -878527175         
30  6    70   changqia  224190018    DONE       -878527175         
31  1    76   ssolomon  1221542610   DONE       -878527175         
32  2    78   ssolomon  1221542610   DONE       -878527175         
33  3    80   ssolomon  1221542610   DONE       -878527175         
34  4    82   ssolomon  1221542610   DONE       -878527175         
35  5    83   ssolomon  1221542610   DONE       -878527175         
36  6    86   ssolomon  1221542610   DONE       -878527175         
37  7    88   ssolomon  1221542610   DONE       -878527175         
38  8    90   ssolomon  1221542610   DONE       -878527175         
39  9    92   ssolomon  1221542610   DONE       -878527175         
40  10   94   ssolomon  1221542610   DONE       -878527175         
41  11   96   ssolomon  1221542610   DONE       -878527175         
42  12   98   ssolomon  1221542610   DONE       -878527175         
43  13   100  ssolomon  1221542610   DONE       -878527175         
44  14   102  ssolomon  1221542610   DONE       -878527175         
45  15   104  ssolomon  1221542610   DONE       -878527175         
46  16   106  ssolomon  1221542610   DONE       -878527175         
47  17   108  ssolomon  1221542610   DONE       -878527175         
48  18   110  ssolomon  1221542610   DONE       -878527175         
49  19   112  ssolomon  1221542610   DONE       -878527175         

                                                                                                                                           dataset  \
0   data15_13TeV.00280231.physics_Main.deriv.DAOD_HIGG5D1.r9264_p3083_p3213_tid11568050_00                                                           
1   data16_13TeV.00300655.physics_Main.deriv.DAOD_HIGG5D1.r9264_p3083_p3372_tid12610361_00                                                           
2   data15_13TeV.00280231.physics_Main.deriv.DAOD_HIGG5D1.r9264_p3083_p3213_tid115680

In [26]:
df_proportion.count()
#ohne join_key_lookup IS NOT NULL: 579934, mit der beschränkung: 578390
# mit : 578390

578390

In [27]:
df_proportion_percent = df_proportion.withColumn("df_percentage", df_proportion['Seq']/df_proportion['count'])
df_proportion_percent.limit(50).toPandas()

Seq   id   account  accountHash clientState  clientStateHash  \
0   1    2    yliu      1494332205   DONE       -878527175         
1   1    4    changqia  224190018    DONE       -878527175         
2   2    7    yliu      1494332205   DONE       -878527175         
3   1    8    heng      99410464     DONE       -878527175         
4   3    9    yliu      1494332205   DONE       -878527175         
5   4    10   yliu      1494332205   DONE       -878527175         
6   5    11   yliu      1494332205   DONE       -878527175         
7   6    14   yliu      1494332205   DONE       -878527175         
8   7    18   yliu      1494332205   DONE       -878527175         
9   2    19   heng      99410464     DONE       -878527175         
10  8    21   yliu      1494332205   DONE       -878527175         
11  9    22   yliu      1494332205   DONE       -878527175         
12  10   23   yliu      1494332205   DONE       -878527175         
13  2    29   changqia  224190018    DONE       -878527175         
14  1    31   ssolomon  1221542610   DONE       -878527175         
15  1    37   ssolomon  1221542610   DONE       -878527175         
16  11   38   yliu      1494332205   DONE       -878527175         
17  12   39   yliu      1494332205   DONE       -878527175         
18  1    41   ssolomon  1221542610   DONE       -878527175         
19  13   42   yliu      1494332205   DONE       -878527175         
20  1    44   ssolomon  1221542610   DONE       -878527175         
21  14   45   yliu      1494332205   DONE       -878527175         
22  15   49   yliu      1494332205   DONE       -878527175         
23  16   50   yliu      1494332205   DONE       -878527175         
24  3    52   heng      99410464     DONE       -878527175         
25  17   53   yliu      1494332205   DONE       -878527175         
26  3    55   changqia  224190018    DONE       -878527175         
27  1    57   yliu      1494332205   DONE       -878527175         
28  4    59   changqia  224190018    DONE       -878527175         
29  5    64   changqia  224190018    DONE       -878527175         
30  6    70   changqia  224190018    DONE       -878527175         
31  1    76   ssolomon  1221542610   DONE       -878527175         
32  2    78   ssolomon  1221542610   DONE       -878527175         
33  3    80   ssolomon  1221542610   DONE       -878527175         
34  4    82   ssolomon  1221542610   DONE       -878527175         
35  5    83   ssolomon  1221542610   DONE       -878527175         
36  6    86   ssolomon  1221542610   DONE       -878527175         
37  7    88   ssolomon  1221542610   DONE       -878527175         
38  8    90   ssolomon  1221542610   DONE       -878527175         
39  9    92   ssolomon  1221542610   DONE       -878527175         
40  10   94   ssolomon  1221542610   DONE       -878527175         
41  11   96   ssolomon  1221542610   DONE       -878527175         
42  12   98   ssolomon  1221542610   DONE       -878527175         
43  13   100  ssolomon  1221542610   DONE       -878527175         
44  14   102  ssolomon  1221542610   DONE       -878527175         
45  15   104  ssolomon  1221542610   DONE       -878527175         
46  16   106  ssolomon  1221542610   DONE       -878527175         
47  17   108  ssolomon  1221542610   DONE       -878527175         
48  18   110  ssolomon  1221542610   DONE       -878527175         
49  19   112  ssolomon  1221542610   DONE       -878527175         

                                                                                                                                           dataset  \
0   data15_13TeV.00280231.physics_Main.deriv.DAOD_HIGG5D1.r9264_p3083_p3213_tid11568050_00                                                           
1   data16_13TeV.00300655.physics_Main.deriv.DAOD_HIGG5D1.r9264_p3083_p3372_tid12610361_00                                                           
2   data15_13TeV.00280231.physics_Main.deriv.DAOD_HIGG5D1.r9264_p3083_p3213_tid115680

In [28]:
df_proportion_percent.createOrReplaceTempView("percent")
spark.sql("SELECT * FROM percent WHERE join_key_lookup IS NULL").toPandas()
#sollte nichts gefunden werden

Empty DataFrame
Columns: [Seq, id, account, accountHash, clientState, clientStateHash, dataset, datasetHash, datasetScope, datasetScopeHash, dataset_version, dataset_versionHash, datatype, datatypeHash, eventType, eventTypeHash, eventVersion, eventVersionHash, fileScope, filename, filenameHash, filesize_double, hits, hostname, hostnameHash, prod_step, prod_stepHash, project, projectHash, protocol, protocolHash, remoteSite, remoteSiteHash, run_number, run_numberHash, scopeHash, stream_name, stream_nameHash, traceId, traceIdHash, traceIp, traceIpHash, traceTimeentryUnix, transferStart, uuid, uuidHash, join_key_april, count, join_key_lookup, row_number, df_percentage]
Index: []

In [29]:
#spark.sql("SELECT * FROM percent WHERE id == 893353199188").toPandas()

In [30]:
df_proportion_percent = spark.sql("SELECT CAST(CEILING(df_percentage*100) AS int) AS int_percentage,* FROM percent")

In [31]:
df_proportion_percent.createOrReplaceTempView("percent")

In [32]:
#spark.sql("SELECT * FROM percent WHERE id == 893353199188").toPandas()

In [33]:
spark.sql("SELECT * FROM percent WHERE join_key_lookup IS NULL").toPandas()

Empty DataFrame
Columns: [int_percentage, Seq, id, account, accountHash, clientState, clientStateHash, dataset, datasetHash, datasetScope, datasetScopeHash, dataset_version, dataset_versionHash, datatype, datatypeHash, eventType, eventTypeHash, eventVersion, eventVersionHash, fileScope, filename, filenameHash, filesize_double, hits, hostname, hostnameHash, prod_step, prod_stepHash, project, projectHash, protocol, protocolHash, remoteSite, remoteSiteHash, run_number, run_numberHash, scopeHash, stream_name, stream_nameHash, traceId, traceIdHash, traceIp, traceIpHash, traceTimeentryUnix, transferStart, uuid, uuidHash, join_key_april, count, join_key_lookup, row_number, df_percentage]
Index: []

In [34]:
spark.sql("SELECT COUNT(*) FROM percent WHERE CEILING(df_percentage*100) == 60 AND count > 1").limit(10).toPandas()
#6347

count(1)
0  6347

In [35]:
spark.sql("SELECT * FROM percent WHERE CEILING(df_percentage*100) == 60 AND count > 1").limit(10).toPandas()

int_percentage  Seq    id   account  accountHash clientState  \
0  60              9    418   bejaeger -479767920    DONE         
1  60              33   561   sutt      1271817927   DONE         
2  60              58   882   rasmith  -1063263902   DONE         
3  60              6    971   pgadow    1992700062   DONE         
4  60              60   1042  pgadow    1992700062   DONE         
5  60              60   1150  pgadow    1992700062   DONE         
6  60              60   1315  pgadow    1992700062   DONE         
7  60              60   1378  pgadow    1992700062   DONE         
8  60              13   1401  pgadow    1992700062   DONE         
9  60              16   1412  pgadow    1992700062   DONE         

   clientStateHash  \
0 -878527175         
1 -878527175         
2 -878527175         
3 -878527175         
4 -878527175         
5 -878527175         
6 -878527175         
7 -878527175         
8 -878527175         
9 -878527175         

                                                                                                                                  dataset  \
0  data15_13TeV.00278748.physics_Main.deriv.DAOD_JETM1.r9264_p3083_p3372_tid12601869_00                                                     
1  data17_13TeV.00339070.physics_EnhancedBias.merge.HIST.r10367_r10369_p3493_tid13550209_00                                                 
2  mc16_13TeV.426131.Sherpa_CT10_jets_JZ1.merge.AOD.e4355_s3126_r9364_r9315_tid11188943_00                                                  
3  mc16_13TeV.410501.PowhegPythia8EvtGen_A14_ttbar_hdamp258p75_nonallhad.deriv.NTUP_PILEUP.e5458_s3126_r10321_r10210_p3384_tid13341303_00   
4  mc16_13TeV.361106.PowhegPythia8EvtGen_AZNLOCTEQ6L1_Zee.deriv.NTUP_PILEUP.e3601_e5984_s3243_r10236_r9778_p3384_tid13015470_00             
5  mc16_13TeV.361106.PowhegPythia8EvtGen_AZNLOCTEQ6L1_Zee.deriv.NTUP_PILEUP.e3601_e5984_s3239_r10236_r9778_p3384_tid13015404_00             
6  mc16_13TeV.361106.PowhegPythia8EvtGen_AZNLOCTEQ6L1_Zee.deriv.NTUP_PILEUP.e3601_e5984_s3242_r10236_r9778_p3384_tid13015457_00             
7  mc16_13TeV.361106.PowhegPythia8EvtGen_AZNLOCTEQ6L1_Zee.deriv.NTUP_PILEUP.e3601_e5984_s3241_r10236_r9778_p3384_tid13015445_00             
8  mc16_13TeV.361104.PowhegPythia8EvtGen_AZNLOCTEQ6L1_Wminusmunu.deriv.NTUP_PILEUP.e3601_s3126_r10337_r9315_p3288_tid13567418_00            
9  mc16_13TeV.361101.PowhegPythia8EvtGen_AZNLOCTEQ6L1_Wplusmunu.deriv.NTUP_PILEUP.e3601_s3126_r10337_r9315_p3288_tid13567618_00             

   datasetHash  datasetScope  datasetScopeHash  \
0 -1730527489   data15_13TeV  1217185487         
1 -981346263    data17_13TeV -841583270          
2 -2016724839   mc16_13TeV   -1447489496         
3 -1961909749   mc16_13TeV   -1447489496         
4 -689606983    mc16_13TeV   -1447489496         
5 -1195535573   mc16_13TeV   -1447489496         
6 -65108818     mc16_13TeV   -1447489496         
7 -68495168     mc16_13TeV   -1447489496         
8 -882194356    mc16_13TeV   -1447489496         
9 -2107222792   mc16_13TeV   -1447489496         

                                       dataset_version  dataset_versionHash  \
0  r9264_p3083_p3372_tid12601869_00                    -1868930008            
1  r10367_r10369_p3493_tid13550209_00                  -916609358             
2  e4355_s3126_r9364_r9315_tid11188943_00               1991089920            
3  e5458_s3126_r10321_r10210_p3384_tid13341303_00       1449943740            
4  e3601_e5984_s3243_r10236_r9778_p3384_tid13015470_00 -202896748             
5  e3601_e5984_s3239_r10236_r9778_p3384_tid13015404_00 -1004252092            
6  e3601_e5984_s3242_r10236_r9778_p3384_tid13015457_00  1254955758            
7  e3601_e5984_s3241_r10236_r9778_p3384_tid13015445_00 -725964590             
8  e3601_s3126_r10337_r9315_p3288_tid13567418_00       -1129943594            
9  e3601_s3126_r10337_r9315_p3288_tid13567618_00        912311260             

      datatype  datatypeHash eventType  eventType

## add maximum reached percentage to each uuid

In [36]:
# sind auch 1%er dabei
spark.sql("SELECT uuid, MAX(int_percentage) FROM percent WHERE count > 1 GROUP BY uuid ORDER BY 2 DESC").limit(250).toPandas()

uuid  max(int_percentage)
0    72ad523cbafe413ca6bf8e7a94b93210  100                
1    2f5cf3eda1564e26963150527623e3a9  100                
2    938de5e1f2624e14a47f173dfb0974a4  100                
3    ad667689bfbd4dda99561f59292e850d  100                
4    56b5f0efc5734d37a18241412994c492  100                
5    09721207300c4e4d97809a8610aa4263  100                
6    e960c4eb31ca444d987626563f9eda89  100                
7    d66aa17ac8994fd2b4d8ee361fd7858e  100                
8    105825f7de48470e82754b0137d6343f  100                
9    dafb94de734e4950a5a985a8ebad1f6d  100                
10   11cf3e52714745c8a759a4fda6d0d592  100                
11   253be7a4ab8e4247931148ec6f8532ea  100                
12   51931ac3f1494f7f8e424789ea7ccf77  100                
13   d01efa5035a243379657f6b4723d11be  100                
14   18de0f27b07f4d2d86e50a9d56f3d57a  100                
15   95a90e3685fe40c1a8f4ca1c2a44688f  100                
16   c8e3998f25ed4765b9148d4f9fd5e25f  100                
17   ed035fc9e36d4f9f9c978c6a664c67f4  100                
18   480f112d26fb444aa1ffb7c1fac5f70d  100                
19   ffbd47cfaeb44807a99dbacf6d604ec0  100                
20   c4fb147a75fa4396994c69d41ba339ed  100                
21   284a8f719c7e4e1291606a9c9106eb3e  100                
22   1ba0049882cc40b799586623085e1ab7  100                
23   18ac1326809545d5bbaf1e245c4c725a  100                
24   7bafcefeb196466db56153d71dab8662  100                
25   aa0631a0d94e43b5aac8a8bdb18703d1  100                
26   f7a4fdfa01794fcdb49bf04c4e5f22f5  100                
27   b7707dfb492742e7b21f677cb45e7c43  100                
28   f54ea3809e054049b495fab7e40da0dd  100                
29   a617cad9a1d04f27bcb91f18d0ccb859  100                
..                                ...  ...                
220  af7e692d4a034427af6e50a75d2cfb83  100                
221  2e08241292c746d681ea22d83b620c11  100                
222  aaea0c60aa434036a80b9129edcc0426  100                
223  8454e7052bee4bc3aab8e50b4977df58  100                
224  5b29345a7db148b58278ea87b0deadd5  100                
225  ed7c311a1b984e9690cb310341ff22d7  100                
226  7674d03301974f609bddbe448afd31cd  100                
227  716bc64e31bf401a85b284e2ee10863f  100                
228  64b24565f5c34c58a13fce6e7d0975c3  100                
229  bba44fb1903b4f418645f7641495c2c7  100                
230  4b128f40166e4b7998e0e4b92411f12b  100                
231  5736e32958b4459585fe6d319af0a420  100                
232  eab31396d1504d3c8a37c11ded8abf31  100                
233  f610eb9d05eb4ea5b9a6e4ba87643836  100                
234  12f1054a1b6041fdac9d47d1704b0e79  100                
235  c7650100d30c4b029881d838f705efd8  100                
236  353935a0546444c9a9e4cecaec11eab7  100                
237  272084db7d2243548f0cd64529073643  100                
238  7bcc74c99c504f4bae62e7e47a12bbca  100                
239  4faf285554cd49f8a93c7386ae021084  100                
240  e0a15169062f4bea86f76aefa3c817c5  100                
241  e6ca73baa3fc4361897b7b6b323e57f8  100                
242  759b396c8ff14fd5aa26611d52192332  100                
243  9693f4dae5984a538d09cbad0a9820e3  100                
244  ec793efcf5c74bd3ab32ceaf211e0442  100                
245  b123eaeca8e146448bb241ccb247279d  100                
246  6e28d8a128bc4bf0a7d66868f9328a43  100                
247  fae836a3b8b34cb983459c9bb2e0755e  100                
248  1a8d8af7c2c74ab7b6091d5f39e29d89  100                
249  7c2516f7d033483da1ed30359f3644ed  100                

[250 rows x 2 columns]

In [37]:
# sind auch 1%er dabei
spark.sql("SELECT max, COUNT(*) FROM (SELECT uuid, MAX(int_percentage) AS max FROM percent WHERE count > 1 GROUP BY uuid ORDER BY 2 DESC) GROUP BY max").limit(250).toPandas()

max  count(1)
0   100  4051    
1   99   9       
2   98   9       
3   97   8       
4   96   2       
5   95   6       
6   94   6       
7   93   7       
8   92   1       
9   91   2       
10  90   7       
11  89   5       
12  88   5       
13  87   2       
14  86   7       
15  85   8       
16  84   9       
17  82   3       
18  81   2       
19  80   11      
20  79   1       
21  78   2       
22  77   3       
23  76   2       
24  75   7       
25  74   2       
26  73   5       
27  72   4       
28  71   1       
29  70   5       
..  ..  ..       
64  30   2       
65  29   5       
66  28   9       
67  27   4       
68  26   1       
69  25   45      
70  24   8       
71  23   6       
72  22   4       
73  21   4       
74  20   28      
75  19   9       
76  18   2       
77  17   25      
78  16   6       
79  15   32      
80  14   8       
81  13   17      
82  12   22      
83  11   11      
84  10   44      
85  9    23      
86  8    35      
87  7    24      
88  6    35      
89  5    42      
90  4    57      
91  3    94      
92  2    185     
93  1    813     

[94 rows x 2 columns]

In [38]:
spark.sql("SELECT * FROM percent WHERE uuid == '72c49c0c0e1a459a86dc85e45fe546d7' AND count > 1 AND dataset=='mc16_13TeV.363632.MGPy8EG_N30NLO_Wmunu_Ht140_280_BFilter.merge.NTUP_PILEUP.e4944_e5984_s3126_r9364_r9315_p3288_p3126_tid12826358_00' ORDER BY traceTimeentryUnix ASC").toPandas()

int_percentage  Seq      id   account  accountHash clientState  \
0  50              1    505300  alaperto  1854766604   DONE         
1  100             2    505302  alaperto  1854766604   DONE         

   clientStateHash  \
0 -878527175         
1 -878527175         

                                                                                                                               dataset  \
0  mc16_13TeV.363632.MGPy8EG_N30NLO_Wmunu_Ht140_280_BFilter.merge.NTUP_PILEUP.e4944_e5984_s3126_r9364_r9315_p3288_p3126_tid12826358_00   
1  mc16_13TeV.363632.MGPy8EG_N30NLO_Wmunu_Ht140_280_BFilter.merge.NTUP_PILEUP.e4944_e5984_s3126_r9364_r9315_p3288_p3126_tid12826358_00   

   datasetHash datasetScope  datasetScopeHash  \
0  682508441    mc16_13TeV  -1447489496         
1  682508441    mc16_13TeV  -1447489496         

                                            dataset_version  \
0  e4944_e5984_s3126_r9364_r9315_p3288_p3126_tid12826358_00   
1  e4944_e5984_s3126_r9364_r9315_p3288_p3126_tid12826358_00   

   dataset_versionHash     datatype  datatypeHash eventType  eventTypeHash  \
0 -111191369            NTUP_PILEUP  1967069222    download  59378080        
1 -111191369            NTUP_PILEUP  1967069222    download  59378080        

  eventVersion  eventVersionHash   fileScope  \
0  1.14.11     -2044360925        mc16_13TeV   
1  1.14.11     -2044360925        mc16_13TeV   

                                   filename  filenameHash  filesize_double  \
0  NTUP_PILEUP.12826358._000001.pool.root.1  1272620512    51980.0           
1  NTUP_PILEUP.12826358._000002.pool.root.1 -1764054282    14514.0           

   hits           hostname  hostnameHash prod_step  prod_stepHash     project  \
0  5     lxplus065.cern.ch  922885429     merge     874345773      mc16_13TeV   
1  5     lxplus065.cern.ch  922885429     merge     874345773      mc16_13TeV   

   projectHash protocol  protocolHash          remoteSite  remoteSiteHash  \
0 -1447489496   gsiftp   2128034789    CERN-PROD_DATADISK -2057286004       
1 -1447489496   gsiftp   2128034789    CERN-PROD_DATADISK -2057286004       

  run_number  run_numberHash   scopeHash  \
0  363632     336230953      -1447489496   
1  363632     336230953      -1447489496   

                              stream_name  stream_nameHash  \
0  MGPy8EG_N30NLO_Wmunu_Ht140_280_BFilter -740908205         
1  MGPy8EG_N30NLO_Wmunu_Ht140_280_BFilter -740908205         

                            traceId  traceIdHash                traceIp  \
0  4d23e3a682d4473fb1e9f7a10c7f66d8  795147204    ::ffff:137.138.52.127   
1  71f0d528720546809c12bd4ce44e76a5 -422568853    ::ffff:137.138.52.127   

   traceIpHash  traceTimeentryUnix  transferStart  \
0  438086758    1.523876e+09        1.523876e+09    
1  438086758    1.523876e+09        1.523876e+09    

                               uuid    uuidHash  \
0  72c49c0c0e1a459a86dc85e45fe546d7 -1055568383   
1  72c49c0c0e1a459a86dc85e45fe546d7 -1055568383   

                                                                                                                                   join_key_april  \
0  mc16_13TeV:mc16_13TeV.363632.MGPy8EG_N30NLO_Wmunu_Ht140_280_BFilter.merge.NTUP_PILEUP.e4944_e5984_s3126_r9364_r9315_p3288_p3126_tid12826358_00   
1  mc16_13TeV:mc16_13TeV.363632.MGPy8EG_N30NLO_Wmunu_Ht140_280_BFilter.merge.NTUP_PILEUP.e4944_e5984_s3126_r9364_r9315_p3288_p3126_tid12826358_00   

   count  \
0  2       
1  2       

                                                                                                                                  join_key_lookup  \
0  mc16_13TeV:mc16_13TeV.363632.MGPy8EG_N30NLO_Wmunu_Ht140_280_BFilter.merge.NTUP_PILEUP.e4944_e5984_s3126_r9364_r9315_p3288_p3126_tid12826358_00   
1  mc16_13TeV:mc16_13TeV.363632.MGPy8EG_N30NLO_Wmunu_Ht140_280_BFilter.merge.NTUP_PILEUP.e4944_e5984_s3126_r9364_r9315_p3288_p3126_tid12826358_00   

   row_number  df_percentage  
0  1           0.5            
1  1           1.0

## List all uuids that go over 1 hour time windows

In [39]:
spark.sql("SELECT uuid,(max-min)/3600 AS diff_hours, from_unixtime(min) AS min, from_unixtime(max) AS max \
           FROM(SELECT uuid, MIN(traceTimeentryUnix) AS min, MAX(traceTimeentryunix) AS max\
           FROM percent WHERE count > 1 GROUP BY uuid)\
           WHERE min != max AND (max-min)/3600 > 1.0\
           ORDER BY diff_hours DESC").toPandas()

uuid  diff_hours                  min  \
0    50288de2738a423f9b650f404e4da890  144.461958  2018-04-08 05:06:28   
1    606aa9f5a21348d2a0eeb660198d6600  144.314361  2018-04-01 03:39:19   
2    bb4fcae1179e4ca89e2796e5a5fc3d65  103.874410  2018-04-23 01:46:26   
3    8b6fef65016d4bf08f15a0e861d4bad3  101.066326  2018-04-19 00:45:15   
4    24390937bd0a4050900c700a727650ea  96.037586   2018-04-12 18:46:43   
5    8baa77fc5c2a48e1a555ad11c06d8553  95.907414   2018-04-18 11:54:53   
6    17b1f1dedbac482397aea75d7f7d1c94  94.384630   2018-04-18 13:35:56   
7    3f2275aadc1b409abef8a0bb41eaf607  87.329864   2018-04-18 02:34:48   
8    4601b69722da41a189695ad839482940  74.660284   2018-04-27 20:56:46   
9    dda87a65787147a9ba32f15f00ff6082  71.870229   2018-04-11 10:31:26   
10   56075424a6d54596b00bd491cadb0dc3  62.669091   2018-04-21 18:04:04   
11   f281c6b71d904302949c11f97d2aee03  58.589340   2018-04-24 08:50:21   
12   b463267f12de4c4f98654c169358cdfd  52.909110   2018-04-08 09:43:23   
13   3e15cf0a38a34d72b185c19756954d31  50.122993   2018-04-06 15:33:54   
14   34868ec92d144675b32f528e6d55afc4  48.306215   2018-04-12 13:31:19   
15   4dbc7ffda6e04ce1a835836b664d159c  48.190240   2018-04-12 16:08:19   
16   a49c0353bc584e83abbc55ece74b16af  45.889140   2018-04-12 18:13:57   
17   68cb93eb562a427e91bef1e617338fb8  45.428653   2018-04-19 05:52:47   
18   43ae524f8eb049e99d48dd6089f59599  44.554986   2018-04-15 21:19:36   
19   7f39ec4282594aff9933ce6b95c8e240  44.213854   2018-04-15 21:38:42   
20   523e6c129a8245519f3c10e9b7ff35f8  43.053974   2018-04-01 01:48:42   
21   658f9f7f8bd54a689fb810f337ae3ad8  43.035997   2018-04-19 08:16:33   
22   b3250112288b442586b30738ae6b37f2  41.804424   2018-04-12 20:54:41   
23   3834b45cac2840b7a4d28c5f559d888e  37.194909   2018-04-10 17:07:02   
24   d654b67d22794069b08baf57f6aea3aa  37.160429   2018-04-10 17:05:04   
25   895eaa69307d444d91d1d38a26629f83  37.064493   2018-04-21 06:23:02   
26   5c8382c8bd894d34a295c07ce886b474  34.659995   2018-04-12 20:55:39   
27   03cbfe4b769a464d97690e2a5f4a4e77  33.928824   2018-04-29 14:04:02   
28   6f10d274f08e42e6a9997376982db6c2  32.522593   2018-04-15 22:05:15   
29   10fb2d9f030140c5b5e262fb85ee4d48  31.379725   2018-04-27 06:24:56   
..                                ...        ...                   ...   
488  5811283f3eff4cf48a018e2ecfe38db9  1.107532    2018-04-01 23:40:02   
489  1e88645488d34898969082800451c978  1.107455    2018-04-01 22:35:13   
490  21a39462d62a429d902029fe0e6f7717  1.102230    2018-04-25 18:43:58   
491  5138de77b4de4bf2922f8416040bb045  1.100930    2018-04-19 14:31:36   
492  ee3917ff152048e4a416a4118383d00a  1.093341    2018-04-07 13:22:53   
493  61f786b12629402c92ea27ce7ced88f2  1.091452    2018-04-22 15:59:28   
494  e541909f1a794c6fa99350c03ec64932  1.086079    2018-04-11 21:34:45   
495  c17d2f91c0af489fad2cbe1571d2da98  1.085941    2018-04-12 17:27:40   
496  7199922edbec491e8fdfb6f7c9984947  1.085247    2018-04-11 22:39:14   
497  aa08c17c2ce04b7ab8ba27d4f1a798fe  1.074163    2018-04-05 22:27:14   
498  8d507ceb54dc4d64b740df076e2ef972  1.073923    2018-04-11 21:52:32   
499  4ac23da1f70344df90820882182b0214  1.072041    2018-04-04 04:07:19   
500  73eb2a9ac54246ffac52a181f27bad29  1.070081    2018-04-11 19:26:11   
501  ba480ce7970e4d7e8c5f9bd03887d3b8  1.065958    2018-04-26 14:33:49   
502  e39383850f0c47cea7708c2dc0381a69  1.061250    2018-04-11 17:05:15   
503  44986a4afa9b4287a13de03d7a176ef0  1.057270    2018-04-03 10:58:38   
504  e3428d084ade49978c790f19943caaa4  1.056287    2018-04-11 21:37:14   
505  dca5af3be87147ce99a66d5865dcd926  1.052336    2018-04-18 04:15:05   
506  c8cb1fe0e29f4e7484c4aa4c1cfa6432  1.051696    2018-04-16 00:18:43   
507  ac254208dc53460ea36f52b06f001303  1.048785    2018-04-05 12:31:37   
508  3d6bb0451adc4b9eb90cac9616afb36d  1.046991    2018-04-05 20:10:21   
509  c549db7cb0bf4b5ca5f588a7c2fb2600  1.043964    2018-04-11 20:10:32   
510  769a50d7ddd34407a1d8391ead535a80

In [40]:
spark.sql("SELECT * FROM percent WHERE uuid == '50288de2738a423f9b650f404e4da890' AND count > 1 ORDER BY int_percentage ASC").toPandas()

int_percentage  Seq      id account  accountHash clientState  \
0    1               1    149241  yliu    1494332205   DONE         
1    1               2    149256  yliu    1494332205   DONE         
2    1               3    149397  yliu    1494332205   DONE         
3    1               4    149929  yliu    1494332205   DONE         
4    1               5    149968  yliu    1494332205   DONE         
5    1               6    149995  yliu    1494332205   DONE         
6    1               7    150106  yliu    1494332205   DONE         
7    1               8    150139  yliu    1494332205   DONE         
8    1               9    150181  yliu    1494332205   DONE         
9    1               10   150779  yliu    1494332205   DONE         
10   1               11   151261  yliu    1494332205   DONE         
11   1               12   152330  yliu    1494332205   DONE         
12   2               13   152604  yliu    1494332205   DONE         
13   2               14   153436  yliu    1494332205   DONE         
14   2               15   161611  yliu    1494332205   DONE         
15   2               16   161655  yliu    1494332205   DONE         
16   2               17   161673  yliu    1494332205   DONE         
17   2               18   161679  yliu    1494332205   DONE         
18   2               19   161687  yliu    1494332205   DONE         
19   2               20   163726  yliu    1494332205   DONE         
20   2               21   164089  yliu    1494332205   DONE         
21   2               22   164533  yliu    1494332205   DONE         
22   2               23   166162  yliu    1494332205   DONE         
23   2               24   166619  yliu    1494332205   DONE         
24   3               25   166969  yliu    1494332205   DONE         
25   3               26   167342  yliu    1494332205   DONE         
26   3               27   170878  yliu    1494332205   DONE         
27   3               28   172547  yliu    1494332205   DONE         
28   3               29   172611  yliu    1494332205   DONE         
29   3               30   173165  yliu    1494332205   DONE         
..  ..               ..      ...   ...           ...    ...         
191  16              192  443086  yliu    1494332205   DONE         
192  16              193  443546  yliu    1494332205   DONE         
193  16              194  444281  yliu    1494332205   DONE         
194  16              195  446689  yliu    1494332205   DONE         
195  17              196  447128  yliu    1494332205   DONE         
196  17              197  447503  yliu    1494332205   DONE         
197  17              198  450704  yliu    1494332205   DONE         
198  17              199  451020  yliu    1494332205   DONE         
199  17              200  451278  yliu    1494332205   DONE         
200  17              201  452155  yliu    1494332205   DONE         
201  17              202  452435  yliu    1494332205   DONE         
202  17              203  452881  yliu    1494332205   DONE         
203  17              204  458858  yliu    1494332205   DONE         
204  17              205  459143  yliu    1494332205   DONE         
205  17              206  461359  yliu    1494332205   DONE         
206  17              207  462580  yliu    1494332205   DONE         
207  18              208  463093  yliu    1494332205   DONE         
208  18              209  463607  yliu    1494332205   DONE         
209  18              210  464450  yliu    1494332205   DONE         
210  18              211  464979  yliu    1494332205   DONE         
211  18              212  465039  yliu    1494332205   DONE         
212  18              213  469208  yliu    1494332205   DONE         
213  18              214  469498  yliu    1494332205   DONE         
214  18              215  470014  yliu    1494332205   DONE         
215  18              216  471294  yliu    1494332205   DONE         
216  18              217  472766  yliu    1494332205   DONE         
217

In [41]:
df_proportion_percent.createOrReplaceTempView("percent")

In [42]:
spark.sql("SELECT * FROM percent WHERE join_key_lookup IS NULL").toPandas()

Empty DataFrame
Columns: [int_percentage, Seq, id, account, accountHash, clientState, clientStateHash, dataset, datasetHash, datasetScope, datasetScopeHash, dataset_version, dataset_versionHash, datatype, datatypeHash, eventType, eventTypeHash, eventVersion, eventVersionHash, fileScope, filename, filenameHash, filesize_double, hits, hostname, hostnameHash, prod_step, prod_stepHash, project, projectHash, protocol, protocolHash, remoteSite, remoteSiteHash, run_number, run_numberHash, scopeHash, stream_name, stream_nameHash, traceId, traceIdHash, traceIp, traceIpHash, traceTimeentryUnix, transferStart, uuid, uuidHash, join_key_april, count, join_key_lookup, row_number, df_percentage]
Index: []

In [43]:
df_proportion_percent_dropped = df_proportion_percent.selectExpr("id as id_proportion", "int_percentage")
df_proportion_percent_dropped.limit(10).toPandas()

id_proportion  int_percentage
0  2              1             
1  4              2             
2  7              1             
3  8              3             
4  9              2             
5  10             2             
6  11             2             
7  14             3             
8  18             3             
9  19             5

In [44]:
#df_final = df_joined_indexed.join(df_proportion_percent_dropped, df_joined_indexed.id == df_proportion_percent_dropped.id_proportion, "left_outer")
df_joined_indexed.createOrReplaceTempView("joined_indexed")
df_proportion_percent_dropped.createOrReplaceTempView("percent_dropped")
df_final = spark.sql("SELECT * FROM joined_indexed LEFT JOIN percent_dropped ON joined_indexed.id == percent_dropped.id_proportion")

In [45]:
df_final.createOrReplaceTempView("final")
spark.sql("SELECT * FROM final WHERE join_key_lookup IS NULL").toPandas()

id   account  accountHash         clientState  clientStateHash  \
0     834     drafanoh -33750993     DONE               -878527175         
1     835     drafanoh -33750993     DONE               -878527175         
2     836     drafanoh -33750993     DONE               -878527175         
3     837     drafanoh -33750993     DONE               -878527175         
4     838     drafanoh -33750993     DONE               -878527175         
5     840     drafanoh -33750993     DONE               -878527175         
6     843     drafanoh -33750993     DONE               -878527175         
7     844     drafanoh -33750993     DONE               -878527175         
8     845     drafanoh -33750993     DONE               -878527175         
9     846     drafanoh -33750993     DONE               -878527175         
10    847     drafanoh -33750993     DONE               -878527175         
11    848     drafanoh -33750993     ServiceUnavailable -1394468120        
12    850     drafanoh -33750993     DONE               -878527175         
13    852     drafanoh -33750993     DONE               -878527175         
14    853     drafanoh -33750993     DONE               -878527175         
15    55952   pgadow    1992700062   DONE               -878527175         
16    59972   drafanoh -33750993     DONE               -878527175         
17    59975   drafanoh -33750993     DONE               -878527175         
18    59977   drafanoh -33750993     DONE               -878527175         
19    154943  jojungge -1078508397   DONE               -878527175         
20    154978  jojungge -1078508397   DONE               -878527175         
21    155362  jojungge -1078508397   DONE               -878527175         
22    155374  jojungge -1078508397   DONE               -878527175         
23    158599  jojungge -1078508397   DONE               -878527175         
24    159553  jojungge -1078508397   DONE               -878527175         
25    159558  jojungge -1078508397   DONE               -878527175         
26    159559  jojungge -1078508397   DONE               -878527175         
27    159596  jojungge -1078508397   DONE               -878527175         
28    159608  jojungge -1078508397   DONE               -878527175         
29    159640  jojungge -1078508397   DONE               -878527175         
...      ...       ...         ...    ...                      ...         
1594  707190  lcorpe   -279559867    DONE               -878527175         
1595  707201  lcorpe   -279559867    DONE               -878527175         
1596  707217  lcorpe   -279559867    DONE               -878527175         
1597  707233  lcorpe   -279559867    DONE               -878527175         
1598  707242  lcorpe   -279559867    DONE               -878527175         
1599  707254  lcorpe   -279559867    DONE               -878527175         
1600  709118  lcorpe   -279559867    DONE               -878527175         
1601  709202  mcfayden -1592538155   DONE               -878527175         
1602  726095  schsu    -927799723    ServiceUnavailable -1394468120        
1603  726120  schsu    -927799723    DONE               -878527175         
1604  735502  goblirsc -305354976    DONE               -878527175         
1605  745375  khamano  -1185418172   DONE               -878527175         
1606  745409  khamano  -1185418172   DONE               -878527175         
1607  745427  khamano  -1185418172   DONE               -878527175         
1608  745436  khamano  -1185418172   DONE               -878527175         
1609  745471  khamano  -1185418172   DONE               -878527175         
1610  745558  khamano  -1185418172   DONE               -878527175         
1611  745567  khamano  -1185418172   DONE               -878527175         
1612  745577  khamano  -1185418172   DONE               -878527175         
1613  745601  khamano  -1185418172   DONE               -878527175         
1614  766780  hegormse -2036406190   DONE               -8

In [46]:
df_final.count()

777286

In [47]:
df_final.sort("traceTimeentryUnix", ascending=True).limit(10).toPandas()

id   account  accountHash         clientState  clientStateHash  \
0  1   yliu      1494332205   ServiceUnavailable -1394468120        
1  2   yliu      1494332205   DONE               -878527175         
2  3   yliu      1494332205   ServiceUnavailable -1394468120        
3  4   changqia  224190018    DONE               -878527175         
4  5   changqia  224190018    ServiceUnavailable -1394468120        
5  6   yliu      1494332205   ServiceUnavailable -1394468120        
6  7   yliu      1494332205   DONE               -878527175         
7  8   heng      99410464     DONE               -878527175         
8  9   yliu      1494332205   DONE               -878527175         
9  10  yliu      1494332205   DONE               -878527175         

                                                                                                                                          dataset  \
0  data15_13TeV.00280231.physics_Main.deriv.DAOD_HIGG5D1.r9264_p3083_p3213_tid11568050_00                                                           
1  data15_13TeV.00280231.physics_Main.deriv.DAOD_HIGG5D1.r9264_p3083_p3213_tid11568050_00                                                           
2  data15_13TeV.00280231.physics_Main.deriv.DAOD_HIGG5D1.r9264_p3083_p3213_tid11568050_00                                                           
3  data16_13TeV.00300655.physics_Main.deriv.DAOD_HIGG5D1.r9264_p3083_p3372_tid12610361_00                                                           
4  data16_13TeV.00300655.physics_Main.deriv.DAOD_HIGG5D1.r9264_p3083_p3372_tid12610361_00                                                           
5  data15_13TeV.00280231.physics_Main.deriv.DAOD_HIGG5D1.r9264_p3083_p3213_tid11568050_00                                                           
6  data15_13TeV.00280231.physics_Main.deriv.DAOD_HIGG5D1.r9264_p3083_p3213_tid11568050_00                                                           
7  mc16_13TeV.345056.PowhegPythia8EvtGen_NNPDF3_AZNLO_ZH125J_MINLO_vvbb_VpT.deriv.DAOD_HIGG5D1.e5706_e5984_s3126_r9364_r9315_p3260_tid12139571_00   
8  data15_13TeV.00280231.physics_Main.deriv.DAOD_HIGG5D1.r9264_p3083_p3213_tid11568050_00                                                           
9  data15_13TeV.00280231.physics_Main.deriv.DAOD_HIGG5D1.r9264_p3083_p3213_tid11568050_00                                                           

   datasetHash  datasetScope  datasetScopeHash  \
0 -477538451    data15_13TeV  1217185487         
1 -477538451    data15_13TeV  1217185487         
2 -477538451    data15_13TeV  1217185487         
3 -654722309    data16_13TeV -915656957          
4 -654722309    data16_13TeV -915656957          
5 -477538451    data15_13TeV  1217185487         
6 -477538451    data15_13TeV  1217185487         
7  3665910      mc16_13TeV   -1447489496         
8 -477538451    data15_13TeV  1217185487         
9 -477538451    data15_13TeV  1217185487         

                                      dataset_version  dataset_versionHash  \
0  r9264_p3083_p3213_tid11568050_00                   -762138301             
1  r9264_p3083_p3213_tid11568050_00                   -762138301             
2  r9264_p3083_p3213_tid11568050_00                   -762138301             
3  r9264_p3083_p3372_tid12610361_00                    2032569103            
4  r9264_p3083_p3372_tid12610361_00                    2032569103            
5  r9264_p3083_p3213_tid11568050_00                   -762138301             
6  r9264_p3083_p3213_tid11568050_00                   -762138301             
7  e5706_e5984_s3126_r9364_r9315_p3260_tid12139571_00  747979904             
8  r9264_p3083_p3213_tid11568050_00                   -762138301             
9  r9264_p3083_p3213_tid11568050_00                   -762138301             

       datatype  datatypeHash eventType  eventTypeHash eventVersion  \
0  DAOD_HIGG5D1  1133870593    download  59378080       1.14.11       
1  DAOD_HIGG5D1  1133870593    download  59378080  

In [48]:
df_final = df_final.sort("traceTimeentryUnix", ascending=True)

In [49]:
df_final.write.format("json").save("/user/sprenner/training_official_april_proportion.json")
#df_final.write.csv('/user/sprenner/training_official_april_proportion.csv', sep='\t')